In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from sklearn.metrics import mean_squared_error

train_df = pd.read_csv("train.csv", parse_dates=["date"])
test_df  = pd.read_csv("test.csv",  parse_dates=["date"])

def basic_fe(df):
    df = df.copy()
    d  = df["date"]
    df["day_of_week"]  = d.dt.dayofweek
    df["month"]        = d.dt.month
    df["year"]         = d.dt.year
    df["is_weekend"]   = (df["day_of_week"] >= 5).astype(int)
    df["temp_mean"]          = (df["temperature_2m_max"] + df["temperature_2m_min"]) / 2
    df["tempmax_minus_evap"] = df["temperature_2m_max"] - df["et0_fao_evapotranspiration"]
    df["rad_x_tempmax"]      = df["shortwave_radiation_sum"] * df["temperature_2m_max"]
    df["wind_avg"]           = (df["wind_gusts_10m_max"] + df["wind_speed_10m_max"]) / 2
    return df

train_df = basic_fe(train_df)
test_df  = basic_fe(test_df)

def add_cid(df):
    df = df.copy()
    df["cid"] = df["cluster_id"].astype(str).str.extract(r"(\d+)")[0].astype(int)
    return df

train_df = add_cid(train_df)
test_df  = add_cid(test_df)

ROLL_COLS = ["temperature_2m_max", "daylight_duration", "temp_mean", "apparent_temperature_min"]
train_df = train_df.sort_values(["cid", "date"]).copy()
for col in ROLL_COLS:
    train_df[f"delta_{col}"]    = train_df.groupby("cid")[col].diff()
    train_df[f"rolling7_{col}"] = train_df.groupby("cid")[col].shift(1).rolling(7, min_periods=1).mean().reset_index(0, drop=True)

combo = pd.concat(
    [train_df[["cid", "date"] + ROLL_COLS],
     test_df[["cid", "date"] + ROLL_COLS]],
    ignore_index=True
).sort_values(["cid", "date"])

for col in ROLL_COLS:
    combo[f"delta_{col}"]    = combo.groupby("cid")[col].diff()
    combo[f"rolling7_{col}"] = combo.groupby("cid")[col].shift(1).rolling(7, min_periods=1).mean().reset_index(0, drop=True)

test_df = test_df.merge(
    combo.loc[combo.index.isin(test_df.index),
              ["cid", "date"] +
              [f"delta_{c}" for c in ROLL_COLS] +
              [f"rolling7_{c}" for c in ROLL_COLS]],
    on=["cid", "date"],
    how="left"
)

all_df = pd.concat([train_df, test_df], sort=False).reset_index(drop=True)
all_df = pd.get_dummies(all_df, columns=["cluster_id"], prefix="cluster")

m = all_df["month"]
all_df["season_phase"] = ((m - 1) % 12) / 12.0
BASE_CDD = 22
all_df["CDD"] = (all_df["temp_mean"] - BASE_CDD).clip(lower=0)
eps = 1e-6
all_df["rad_intensity"] = all_df["shortwave_radiation_sum"] / (all_df["sunshine_duration"] + eps)
all_df["sunshine_ratio"] = all_df["sunshine_duration"] / (all_df["daylight_duration"] + eps)
all_df["wind_dir_rad"] = np.deg2rad(all_df["wind_direction_10m_dominant"])
all_df["wind_dir_sin"] = np.sin(all_df["wind_dir_rad"])
all_df["wind_dir_cos"] = np.cos(all_df["wind_dir_rad"])
all_df["season_cos"] = np.cos(2 * np.pi * (m - 1) / 12)
all_df["seasoncos_x_tempmean"] = all_df["season_cos"] * all_df["temp_mean"]
all_df["seasoncos_x_radiation"] = all_df["season_cos"] * all_df["shortwave_radiation_sum"]
all_df["delta_temp_mean"] = all_df.groupby("cid")["temp_mean"].diff()
all_df["weekend_x_tempmean"] = all_df["is_weekend"] * all_df["temp_mean"]
all_df["seasoncos_x_evap"] = all_df["season_cos"] * all_df["et0_fao_evapotranspiration"]

for cid in all_df["cid"].unique():
    all_df[f"seasoncos_cid{cid}"] = (all_df["cid"]==cid).astype(int) * all_df["season_cos"]
    all_df[f"tempmean_cid{cid}"] = (all_df["cid"] == cid).astype(int) * all_df["temp_mean"]
    all_df[f"rad_cid{cid}"] = (all_df["cid"] == cid).astype(int) * all_df["shortwave_radiation_sum"]
    all_df[f"evap_cid{cid}"] = (all_df["cid"] == cid).astype(int) * all_df["et0_fao_evapotranspiration"]

m = all_df["month"]
all_df["season_cos1"] = np.cos(2 * np.pi * (m - 1) / 12)
all_df["season_cos2"] = np.cos(4 * np.pi * (m - 1) / 12)
all_df["season_cos3"] = np.cos(6 * np.pi * (m - 1) / 12)

for cid in all_df["cid"].unique():
    mask = (all_df["cid"] == cid).astype(int)
    all_df[f"wind_speed_cid{cid}"] = mask * all_df["wind_speed_10m_max"]
    all_df[f"wind_gust_cid{cid}"] = mask * all_df["wind_gusts_10m_max"]
    all_df[f"radintensity_cid{cid}"] = mask * all_df["rad_intensity"]

for cid in all_df["cid"].unique():
    if cid == 3:
        all_df[f"season_cos1_cid{cid}"] = (all_df["cid"]==cid).astype(int) * all_df["season_cos1"]
        all_df[f"season_cos2_cid{cid}"] = (all_df["cid"]==cid).astype(int) * all_df["season_cos2"]
        all_df[f"season_cos3_cid{cid}"] = (all_df["cid"]==cid).astype(int) * all_df["season_cos3"]
    else:
        all_df[f"season_cos1_cid{cid}"] = 0
        all_df[f"season_cos2_cid{cid}"] = 0
        all_df[f"season_cos3_cid{cid}"] = 0

def get_v_shape_feature(month_series):
    return month_series.apply(lambda m: min(abs(m - 8), 12 - abs(m - 8)))

all_df['seasonal_v_shape'] = get_v_shape_feature(all_df['month'])

for cid in all_df["cid"].unique():
    mask = (all_df["cid"] == cid)
    all_df[f"v_shape_cid{cid}"] = all_df['seasonal_v_shape'] * mask.astype(int)

for cid in all_df["cid"].unique():
    mask = (all_df["cid"] == cid).astype(int)
    all_df[f"apparent_tempmin_cid{cid}"] = mask * all_df["apparent_temperature_min"]
    all_df[f"apparent_tempmax_cid{cid}"] = mask * all_df["apparent_temperature_max"]
    all_df[f"tempmax_minus_evap_cid{cid}"] = mask * all_df["tempmax_minus_evap"]
    all_df[f"rad_x_tempmax_cid{cid}"] = mask * all_df["rad_x_tempmax"]

m = all_df["month"]
all_df["is_winter"] = ((m == 12) | (m <= 2)).astype(int)
all_df["is_spring"] = ((m >= 3) & (m <= 5)).astype(int)
all_df["is_summer"] = ((m >= 6) & (m <= 8)).astype(int)
all_df["is_autumn"] = ((m >= 9) & (m <= 11)).astype(int)
all_df["season_quarter"] = ((m - 1) // 3).astype(int)
all_df["season_sin"] = np.sin(2 * np.pi * (m - 1) / 12)

def jan_peak_tri(month):
    dist = np.minimum(abs(month - 8), 12 - abs(month - 8))
    return 1 - dist / 6.0

all_df["jan_peak_tri"] = m.apply(jan_peak_tri)
BASE_HDD, BASE_CDD = 18, 22
all_df["HDD"] = (BASE_HDD - all_df["temp_mean"]).clip(lower=0)
all_df["CDD"] = (all_df["temp_mean"] - BASE_CDD).clip(lower=0)



all_df["wind_chill"] = (
    13.12 + 0.6215 * all_df["temp_mean"]
    - 11.37 * np.power(all_df["wind_avg"], 0.16)
    + 0.3965 * all_df["temp_mean"] * np.power(all_df["wind_avg"], 0.16)
)
all_df["wind_chill_gap"] = all_df["temp_mean"] - all_df["wind_chill"]
all_df["winter_x_temp"] = all_df["is_winter"] * all_df["temp_mean"]
all_df["summer_x_temp"] = all_df["is_summer"] * all_df["temp_mean"]

new_feats = [
    "is_winter", "is_spring", "is_autumn",
    "season_quarter", "season_sin",
    "jan_peak_tri", "HDD", "CDD",
    "wind_chill", "wind_chill_gap",
    "winter_x_temp", "summer_x_temp"
]

for cid in all_df["cid"].unique():
    mask = (all_df["cid"] == cid).astype(int)
    for f in new_feats:
        all_df[f"{f}_cid{cid}"] = mask * all_df[f]

eps = 1e-6
all_df["temp_amp"] = all_df["temperature_2m_max"] - all_df["temperature_2m_min"]
all_df["tdi"] = all_df["apparent_temperature_max"] - all_df["temperature_2m_max"]
all_df["conv_cool"] = all_df["wind_avg"] * (all_df["temperature_2m_max"] - all_df["temp_mean"])
all_df["solar_load"] = all_df["shortwave_radiation_sum"] / (all_df["temp_mean"] + 10 + eps)
all_df["enthalpy_air"] = 1.006 * all_df["temp_mean"]
all_df["moisture_stress"] = all_df["et0_fao_evapotranspiration"] * all_df["temperature_2m_max"]
all_df["rad_temp_quot"] = all_df["rad_intensity"] / (all_df["temp_mean"] + eps)

wx_feats = [
    "temp_amp", "tdi", "conv_cool", "solar_load",
    "enthalpy_air", "moisture_stress", "rad_temp_quot"
]

for cid in all_df["cid"].unique():
    mask = (all_df["cid"] == cid).astype(int)
    for f in wx_feats:
        all_df[f"{f}_cid{cid}"] = mask * all_df[f]

m = all_df["month"]
dist = np.minimum((m - 8).abs(), 12 - (m - 8).abs())
all_df["linear_V"] = 1 - dist/6
all_df["rad_intensity"] = all_df["shortwave_radiation_sum"] / (all_df["sunshine_duration"]+eps)
all_df["solar_load"] = all_df["rad_intensity"] / (all_df["temp_mean"]+10+eps)
all_df["cloudiness"] = 1 - all_df["sunshine_ratio"]
all_df["V_rad_mix"] = all_df["linear_V"] * all_df["shortwave_radiation_sum"]

for cid in all_df["cid"].unique():
    m = (all_df["cid"]==cid).astype(int)
    for f in ["rad_intensity" , "solar_load" , "cloudiness" , "V_rad_mix"]:
        all_df[f"{f}_cid{cid}"] = all_df[f]*m

all_df["HDD_core"] = (18 - all_df["temp_mean"]).clip(lower=0)
all_df["temp_amp"] = all_df["temperature_2m_max"] - all_df["temperature_2m_min"]
all_df["wind_heat_loss"] = all_df["HDD_core"] * all_df["wind_avg"]
all_df["moist_stress"]   = all_df["et0_fao_evapotranspiration"] * all_df["temp_mean"]
all_df["range_per_day"]  = all_df["temp_amp"] / (all_df["daylight_duration"]+eps)

for cid in all_df["cid"].unique():
    m = (all_df["cid"]==cid).astype(int)
    for f in ["wind_heat_loss" , "moist_stress" , "range_per_day"]:
        all_df[f"{f}_cid{cid}"] = all_df[f]*m

drop_cols = ["ID", "date", "electricity_consumption", "wind_direction_10m_dominant", "wind_dir_rad", "HDD_core", "temp_amp" , "is_summer","season_cos1_cid1",
    "season_cos1_cid2",
    "season_cos2_cid1",
    "season_cos2_cid2",
    "season_cos3_cid1",
    "season_cos3_cid2",
    "season_cos1_cid4",
    "season_cos2_cid4","cluster_cluster_1",
    "cluster_cluster_2",
    "cluster_cluster_3",
    "is_winter_cid1",
    "CDD_cid3"]
features = [c for c in all_df.columns if c not in drop_cols and all_df[c].dtype != "object"]

train_final = all_df[all_df["electricity_consumption"].notna()].copy()
test_final  = all_df[all_df["electricity_consumption"].isna()].copy()
window_cols = [c for c in train_final.columns if "delta_" in c or "rolling7_" in c]
train_final = train_final.dropna(subset=window_cols)
train_final = train_final[~train_final["year"].isin([2020])].copy()


y_train = np.log1p(train_final["electricity_consumption"])
X_train = train_final[features]
mask_train = train_final["year"] <= 2017
X_tr, y_tr = X_train[mask_train], y_train[mask_train]
X_val, y_val = X_train[~mask_train], y_train[~mask_train]

lgb_params = dict(
    n_estimators=6000,
    learning_rate=0.02,
    num_leaves=64,
    min_child_samples=80,
    subsample=0.8, subsample_freq=1,
    colsample_bytree=0.7,
    reg_lambda=2.0,
    reg_alpha=1.0,
    random_state=42,
    n_jobs=-1
)

model = lgb.LGBMRegressor(**lgb_params)
model.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    eval_metric="rmse",
    callbacks=[early_stopping(300), log_evaluation(100)]
)

best_iter = model.best_iteration_
full_model = lgb.LGBMRegressor(**{**lgb_params, "n_estimators": best_iter})
full_model.fit(X_train, y_train)

test_pred_log = full_model.predict(test_final[features])
test_final["electricity_consumption"] = np.expm1(test_pred_log)
test_final[["ID", "electricity_consumption"]].to_csv("sub_eleventung222.csv", index=False)
print("✅  sub_eleventung222.csv saved")
print("Jumlah fitur sekarang:", len(features))
